# TẠO CÁC TẬP DỮ LIỆU TRAIN, TEST (SPLITS)

1. Yêu cầu chung: Tạo ra các splits, mỗi split tương ứng với một tập dữ liệu train - test
  + Bài học lý thuyết (để trả lời cho các câu hỏi như vì sao phải cần việc này, thực hiện việc này như thế nào):
    - https://www.kaggle.com/code/satishgunjal/tutorial-k-fold-cross-validation
    - https://machinelearningmastery.com/training-validation-test-split-and-cross-validation-done-right/
    - https://machinelearningmastery.com/how-to-configure-k-fold-cross-validation/

  <IMG SRC = 'https://raw.githubusercontent.com/satishgunjal/images/master/KFold_Cross_Validation.png'>
2. Yêu cầu cụ thể:
- Input:
    + Thư mục cha chứa các thư mục con - mỗi thư mục con tương ứng với tên của từng hiệu xe (Honda, Suzuki, VinFast, Yamaha, Others). Ví dụ: https://drive.google.com/drive/u/1/folders/12WrC9APRzQX36cataTccBiN_Z5F2C0yD
    + Các ảnh được đặt tên theo quy ước: các tập tin ảnh theo quy ước https://colab.research.google.com/drive/1sZBm78OiTUOqbg5-Z2UOKlk3B37TAgd8
    + Số splits NumSplits - mặc định NumSplits=5 (tương đương 5-fold CV)
- Output:
    + File MotocycleDataset.csv - Tập tin chứa tất cả ảnh của dataset
      - Chương trình sẽ scan qua cây thư mục để tìm tất cả các ảnh (chỉ chọn các ảnh có định dạng + phần mở rộng là .jpg)
      - Mỗi dòng sẽ có các thông tin cách nhau bằng dấu phẩy, theo quy ước: ImageFullPath, CategoryID
            - ImageFullPath ở dạng <Thư mục Hiệu xe>/<Tên ảnh>. Ví dụ: Honda/2024123.Honda.1.jpg.
            - CategoryID là số nguyên thuộc [0..5] theo quy ước
              - 0: Others
              - 1: Honda
              - 2: Suzuki
              - 3: Yamaha
              - 4: VinFast     
    + File MotocycleDataset-Splits-[1..5]-[Train/Test].csv - Phân chia thành các splits, mỗi split gồm các ảnh được chia thành thành 2 tập Train - Test
      + Chương trình sẽ đọc dữ liệu từ file  MotocycleDataset.csv, sau đó với mỗi hiệu xe, chia ngẫu nhiên thành 5 tập dữ liệu. Lưu ý là phải chia theo hiệu xe, để đảm bảo dữ liệu Train/Test có dữ liệu của các hiệu xe.
      + Từ 5 tập dữ liệu chia ngẫu nhiên theo các hiệu xe Xij (i là thứ tự tập dữ liệu, j là CategoryID), gom lại thành 5 tập dữ liệu lớn hơn Xi, sao cho mỗi tập dữ liệu này chứa đủ dữ liệu của tất cả các hiệu xe. Tức là Xi = Union(Xij)       
      + Từ 5 tập dữ liệu Xi này (tương ứng với FOLDi ở trong hình vẽ trên), tạo ra 5 splits Split-i
      + Với mỗi bộ dữ liệu Split-i, ghi xuống thành 2 tập tin tương ứng với Train, Test. Ví dụ Split-1 thì ghi thành tập tin  MotocycleDataset-Splits-1-Train.csv và MotocycleDataset-Splits-1-Test.csv. Tập Train gồm 4 bộ, Test gồm bộ còn lại. Ví dụ,
          + Split-1: tập Train sẽ gồm X2, X3, X4, X5, tập Test là X1
          + Split-5: tập Train sẽ gồm X1, X2, X3, X4, tập Test là X5
      + Mỗi dòng sẽ có các thông tin cách nhau bằng dấu phẩy, theo quy ước: ImageFullPath, CategoryID
- Lưu ý:
  - Nên viết thêm các cell
    - Hiển thị danh sách các tên tập tin ảnh trong từng Split-Train/Test,
    - Thống kê các ảnh cho từng CategoryID trong mỗi Split-Train/Test
  - Cần có chú thích
3. Nộp bài: SV share notebook. Các bài nộp sớm sẽ được full điểm. Deadline: 17:00 - 10/06/2024

4. Bài làm đạt yêu cầu sẽ được paste vào notebook với ghi nhận đóng góp từ tác giả.

## Thông tin của tác giả, ngày cập nhật
### Tác giả:
- Phan Thanh Đăng - MSSV: 22520193

### Ngày cập nhật:
- 10/06/2024

## Implementation

### Import libraries and mount drive

In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import StratifiedKFold
import matplotlib.pyplot as plt
import os, csv

from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


### Split K-Fold with NumSplits is 5

In [ ]:
root = 'drive/MyDrive/CS114/Public/'

Category_map = {
  'Others': 0,
  'Honda': 1,
  'Suzuki': 2,
  'Yamaha': 3,
  'VinFast': 4
}

# Numsplits = 5

In [ ]:
# Get all images file from drive/MyDrive/Public/...
def load_data(root, Category_map):
  X_fullpath = []
  y_category = []
  for key, item in Category_map.items():
    path = os.path.join(root, key)
    for dirpath, _, files in os.walk(path):
      for file_name in files:
        if file_name.split('.')[-1].lower() in ['jpg', 'jpeg', 'heic']:
          X_fullpath.append(os.path.join(key, file_name))
          y_category.append(item)

  return X_fullpath, y_category

In [ ]:
def Split_Data(X, y, method, NumSplits=5):
  # MotocycleDataset-Splits-[1..5]-[Train/Test].csv
  cv = method(n_splits=NumSplits, shuffle=True, random_state=42)

  for fold, (train_index, test_index) in enumerate(cv.split(X, y)):
    with open(f'MotocycleDataset-Splits-{fold + 1}-Train.csv', 'w', newline='') as file:
      writer = csv.writer(file)
      for idx in train_index:
        writer.writerow([X[idx], y[idx]])
    with open(f'MotocycleDataset-Splits-{fold + 1}-Test.csv', 'w', newline='') as file:
      writer = csv.writer(file)
      for idx in test_index:
        writer.writerow([X[idx], y[idx]])

In [ ]:
X, y = load_data(root, Category_map)

with open('MotocycleDataset.csv', 'w', newline='') as file:
  writer = csv.writer(file)
  for idx, _ in enumerate(X):
    writer.writerow([X[idx], y[idx]])

Split_Data(X, y, StratifiedKFold)

### Demo result

In [ ]:
def demo(prefix_path, string_number, suffix_path):
  path = prefix_path + string_number + suffix_path + '.csv'
  try:
    df = pd.read_csv(path)
    df.columns = ['Image_path', 'CategoryID']
    # Thống kê các ảnh cho từng CategoryID trong mỗi Split-Train/Test
    if len(string_number) > 0:
      print('Thống kê ảnh cho từng CategoryID:')
      print(df['CategoryID'].value_counts())
    return df
  except FileNotFoundError:
    print("Error")

In [ ]:
demo('MotocycleDataset', '', '')

,Image_path,CategoryID
0,Others/22520896-22520926-22521627.Others.133.jpg,0
1,Others/22520896-22520926-22521627.Others.341.jpg,0
2,Others/22520896-22520926-22521627.Others.55.jpg,0
3,Others/22520896-22520926-22521627.Others.280.jpg,0
4,Others/22520896-22520926-22521627.Others.155.jpg,0
...,...,...
16036,VinFast/21521007-21522629.Vinfast.28.jpg,4
16037,VinFast/21521007-21522629.Vinfast.29.jpg,4
16038,VinFast/21521007-21522629.Vinfast.30.jpg,4
16039,VinFast/21521007-21522629.Vinfast.31.jpg,4


In [ ]:
number = 3
demo('MotocycleDataset-Splits-', str(number) + '-', 'Train')

Thống kê ảnh cho từng CategoryID:
CategoryID
1    4084
3    2543
0    2389
2    2225
4    1592
Name: count, dtype: int64


,Image_path,CategoryID
0,Others/22520896-22520926-22521627.Others.341.jpg,0
1,Others/22520896-22520926-22521627.Others.280.jpg,0
2,Others/22520896-22520926-22521627.Others.155.jpg,0
3,Others/22520896-22520926-22521627.Others.246.jpg,0
4,Others/22520896-22520926-22521627.Others.59.jpg,0
...,...,...
12828,VinFast/21521007-21522629.Vinfast.26.jpg,4
12829,VinFast/21521007-21522629.Vinfast.27.jpg,4
12830,VinFast/21521007-21522629.Vinfast.28.jpg,4
12831,VinFast/21521007-21522629.Vinfast.30.jpg,4


In [ ]:
number = 5
demo('MotocycleDataset-Splits-', str(number) + '-', 'Test')

Thống kê ảnh cho từng CategoryID:
CategoryID
1    1021
3     636
0     596
2     556
4     398
Name: count, dtype: int64


,Image_path,CategoryID
0,Others/22520896-22520926-22521627.Others.280.jpg,0
1,Others/22520896-22520926-22521627.Others.23.jpg,0
2,Others/22520896-22520926-22521627.Others.313.jpg,0
3,Others/22520896-22520926-22521627.Others.255.jpg,0
4,Others/22520896-22520926-22521627.Others.27.jpg,0
...,...,...
3202,VinFast/21521007-21522629.Vinfast.1.jpg,4
3203,VinFast/21521007-21522629.Vinfast.8.jpg,4
3204,VinFast/21521007-21522629.Vinfast.3.jpg,4
3205,VinFast/21521007-21522629.Vinfast.15.jpg,4
